# Projet immo au 27.07.20

> File written by Théo.

## Nombre d'appartements et maisons vendus en 2020

```sql
SELECT sum(total_houses + total_flats) AS sub_total, total_houses, total_flats FROM (
	SELECT * FROM 
	(
		SELECT sum(case when bien.code_type_local = 1 then 1 else 0 end) AS total_houses, 
		sum(case when bien.code_type_local = 2 then 1 else 0 end) AS total_flats
		FROM transaction LEFT JOIN bien ON transaction.bien_id = bien.id 
	) x 
) y GROUP BY total_flats, total_houses;
```
<br>

## Nombre de bien vendu par trimestre

```sql
SELECT 
	sum(case when date_mutation <= '2020-03-31' then 1 else 0 end) as t1,
	sum(case when date_mutation BETWEEN '2020-04-01' AND '2020-06-30' then 1 else 0 end) as t2,
	sum(case when date_mutation BETWEEN '2020-07-01' AND '2020-09-30' then 1 else 0 end) as t3,
	sum(case when date_mutation >= '2020-10-01' then 1 else 0 end) as t4
FROM transaction;
```
<br>

##  Proportion des ventes de biens par trimestre par typologie de bien

```sql
# Expressed as absolute value :
SELECT 
	sum(case when date_mutation <= '2020-03-31' then 1 else 0 end) as t1,
	sum(case when date_mutation BETWEEN '2020-04-01' AND '2020-06-30' then 1 else 0 end) as t2,
	sum(case when date_mutation BETWEEN '2020-07-01' AND '2020-09-30' then 1 else 0 end) as t3,
	sum(case when date_mutation >= '2020-10-01' then 1 else 0 end) as t4
FROM transaction LEFT JOIN bien on transaction.bien_id = bien.id GROUP BY bien.code_type_local;

# Expressed as relative value :
```
<br>

##  Proportion d’appartements vendus par nombre de pièces

```sql
SELECT count(transaction.id), bien.nb_pieces_principales FROM transaction LEFT JOIN bien ON transaction.bien_id = bien.id WHERE bien.code_type_local = 2 GROUP BY bien.nb_pieces_principales ORDER BY bien.nb_pieces_principales ASC;
```
<br>

## Les 10 départements où il y a eu le plus de ventes immobilières

```sql
SELECT count(transaction.id), substr(code_postal, 0, 3) FROM transaction LEFT JOIN bien ON transaction.bien_id = bien.id GROUP BY substr(code_postal, 0, 3) ORDER BY count(transaction.id) DESC LIMIT 10
```
<br>

##  Les 10 départements où il y en a eu le moins

```sql
SELECT count(transaction.id), substr(code_postal, 0, 3) FROM transaction LEFT JOIN bien ON transaction.bien_id = bien.id GROUP BY substr(code_postal, 0, 3) ORDER BY count(transaction.id) ASC LIMIT 10
```
<br>

##  Prix moyen du mètre carré en IDF

```sql
	SELECT (sum(transaction.valeur_fonciere) / sum(bien.surface_reel_bati)) FROM transaction LEFT JOIN bien ON transaction.bien_id = bien.id WHERE substr(bien.code_postal, 0, 3) IN ('75', '92', '93', '94', '78', '77', '91', '95')
```
<br>

##  Liste des 10 appartements les plus chers avec le département et le nombre de mètres carrés

```sql
SELECT * FROM (
	SELECT valeur_fonciere, code_postal, surface_reel_bati FROM transaction LEFT JOIN bien on transaction.bien_id = bien.id ORDER BY transaction.valeur_fonciere DESC
) x WHERE (-1 IN (valeur_fonciere)) IS NOT NULL LIMIT 10
```
<br>

## Taux d’évolution du nombre de ventes entre le premier et le second trimestre de 2020

Evolution rate formula : $ {\displaystyle t={\frac {V_{A}-V_{D}}{|V_{D}|}}} $ where t is expressed in percentage (%)


```sql
SELECT (t2 - t1)::FLOAT / t1 FROM
	(SELECT 
		sum(case when date_mutation <= '2020-03-31' then 1 else 0 end) as t1,
		sum(case when date_mutation BETWEEN '2020-04-01' AND '2020-06-30' then 1 else 0 end) as t2
	FROM transaction) AS ts
```
<br>

##  Liste des communes où le nombre de ventes a augmenté d'au moins 20% entre le premier et le second trimestre de 2020

```sql
SELECT (t2 - t1)::FLOAT/ t1 , commune FROM
	(SELECT 
            sum(case when date_mutation <= '2020-03-31' then 1 else 0 end) as t1,
            sum(case when date_mutation BETWEEN '2020-04-01' AND '2020-06-30' then 1 else 0 end) as t2,
            commune
        FROM transaction
            LEFT JOIN bien ON transaction.bien_id = bien.id
            LEFT JOIN localisation ON bien.code_postal = localisation.code_postal
            LEFT JOIN commune ON localisation.code_commune = commune.code_commune
        GROUP BY commune.commune
	) ts WHERE (t2 - t1)::FLOAT/ t1 >= 0.2 AND commune IS NOT NULL ORDER BY (t2 - t1)::FLOAT/ t1 DESC;
```
<br>